In [84]:
# Import Data 
import datetime
import csv
import copy
import dateutil.parser
import sys
sys.path.insert(1,'../')
import Tools.data_processing as dp
# In the future we should use multiple files for annotating as some anomaly types conflict: 
# Single_point/Plummeting/Skyrocketing/TurbSpike in one file 
# Titled/3Peak/ NormalPeak in another file 
# Peak (reguardless of anomlous status) 
# spikes_fname = '../Data/data_annotating/non_processed_annotated/_.csv'
# anomalous_peaks_fname = '../Data/data_annotating/non_processed_annotated/_.csv'
# normal_peaks_fname = '../Data/data_annotating/non_processed_annotated/_.csv'

fname1 = '/Users/zachfogg/Desktop/DB-SRRW/Data/manual_annotating_data/annotated_data/turb_pp/turb_PP_Labeled_0k-100k-2.csv'
fname2 = '/Users/zachfogg/Desktop/DB-SRRW/Data/manual_annotating_data/annotated_data/turb_pp/turb_PP_Labeled_100k-200k-2.csv'
fname3 = '/Users/zachfogg/Desktop/DB-SRRW/Data/manual_annotating_data/annotated_data/turb_pp/turb_PP_Labeled_200k-300k-2.csv'
# fname1 = '/Users/zachfogg/Desktop/newly_lab_turb_PPP/turb_PPP_0k-100K.csv'
# fname2 = '/Users/zachfogg/Desktop/newly_lab_turb_PPP/turb_PPP_100k-200K.csv'
# fname3 = '/Users/zachfogg/Desktop/newly_lab_turb_PPP/turb_PPP_200k-300K.csv'
# out_file_fname = './data_annotating/processed_annotated/timeseries_0-100000k-annoated.csv'

In [85]:
# Separate data in fDOM, stage, turb 
turb_data = []
stage_data= []
fDOM_data = []
with open(fname1, newline='') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter = ',')
    next(csv_reader,None) # skip headers 
    count = 0
    for row in csv_reader: 
        if count%3 == 0:
            fDOM_data.append([row[1],row[2],row[3]])
        elif count%3 == 1:
            stage_data.append([row[1],row[2],row[3]])
        elif count%3 == 2:
            turb_data.append([row[1],row[2],row[3]])
        count +=1 
    csv_file.close()
with open(fname2, newline='') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter = ',')
    next(csv_reader,None) # skip headers 
    count = 0
    for row in csv_reader: 
        if count%3 == 0:
            fDOM_data.append([row[1],row[2],row[3]])
        elif count%3 == 1:
            stage_data.append([row[1],row[2],row[3]])
        elif count%3 == 2:
            turb_data.append([row[1],row[2],row[3]])
        count +=1 
    csv_file.close()
with open(fname3, newline='') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter = ',')
    next(csv_reader,None) # skip headers 
    count = 0
    for row in csv_reader: 
        if count%3 == 0:
            fDOM_data.append([row[1],row[2],row[3]])
        elif count%3 == 1:
            stage_data.append([row[1],row[2],row[3]])
        elif count%3 == 2:
            turb_data.append([row[1],row[2],row[3]])
        count +=1 
    csv_file.close()

In [86]:
# Check data read
print(len(fDOM_data))
print(len(turb_data))
print(len(stage_data))

229620
229620
229620


In [83]:
turb_data_PPP = copy.deepcopy(turb_data)

In [89]:
# Determine lable names 
discovered = []
for row in turb_data:
    val = row[2]
    if val not in discovered:
        discovered.append(val)
print(discovered)   

['', 'left_base', 'nt_pp', 'right_base', 't_pp', 'sky_dr_st', 't_opp', 'PPP']


In [88]:
# Merge turb PPP and PP data
print(len(turb_data), len(turb_data_PPP))
for i,row in enumerate(turb_data_PPP):
    if row[2] == 'PPP':
        if turb_data[i][2] == 'nt_pp' or turb_data[i][2] == 'sky_dr_st' or turb_data[i][2] == 'PPP':
            turb_data[i][2] = 'PPP'
        else:
            print(turb_data[i],'   ', turb_data_PPP[i])
            break
    elif row[2] == 'NPP' and turb_data[i][2] != 'nt_pp' and turb_data[i][2] != 'sky_dr_st':
        print(turb_data[i],'   ', turb_data_PPP[i])
    elif row[2] == 'PP' and turb_data[i][2] != 't_pp' :
        print(turb_data[i],'   ', turb_data_PPP[i])


229620 229620


In [90]:
turb_data = [[row[0],row[1],row[2],i] for i,row in enumerate(turb_data)]
fDOM_data = [[row[0],row[1],row[2],i] for i,row in enumerate(fDOM_data)]
# Filter for entries that have a label 
fDOM_data_filtered = list(filter(lambda row: row[2] != '', fDOM_data))
turb_data_filtered = list(filter(lambda row: row[2] != '', turb_data))

# Convert ISO timestamp format to python datetime obj and then string
fDOM_data_converted = [[dateutil.parser.isoparse(row[0][:-5]),row[1],row[2],row[3]] for row in fDOM_data_filtered]
turb_data_converted = [[dateutil.parser.isoparse(row[0][:-5]),row[1],row[2],row[3]] for row in turb_data_filtered]

In [91]:
# Map to convert label names
mapLabels = {'t_pp' : 'PP',
             'nt_pp': 'NPP',
             'sky_dr_st' :'NPP',
             'NPP' : 'NPP',
             'PP' :'PP',
             'PPP' :'PPP'} #sky_dr_st should be marked as nt_pp...they can not be caught with current PP rules but will be caught by turb skyrocketing peak
peakLabels = ['t_pp','nt_pp','sky_dr_st','PP','PPP','NPP']
# Write out turb pp lables: [timestamp, indx, label]
out_f = '/Users/zachfogg/Desktop/DB-SRRW/Data/manual_annotating_data/processed_data/turb/julian_time/turb_pp_0k-300k-2_labeled'
# Filter to just pp and npp 
turb_data_pp = list(filter(lambda row: row[2] in peakLabels, turb_data_converted))

with open(out_f, 'w', newline = '') as f: 
    writer = csv.writer(f, delimiter = ',')
    writer.writerow(['timestamp_of_peak', 'value_of_peak','label_of_peak','idx_of_peak'])
    for row in turb_data_pp:
        writer.writerow([dp.datetime_to_julian(row[0]), row[1], mapLabels[row[2]], row[3]])
    f.close()
    
# Write out turb pp lables: [timestamp, indx, label]
out_f = '/Users/zachfogg/Desktop/DB-SRRW/Data/manual_annotating_data/processed_data/turb/datetime/turb_pp_0k-300k-2_labeled'
# Filter to just pp and npp 
with open(out_f, 'w', newline = '') as f: 
    writer = csv.writer(f, delimiter = ',')
    writer.writerow(['timestamp_of_peak', 'value_of_peak','label_of_peak','idx_of_peak'])
    for row in turb_data_pp:
        writer.writerow([row[0],row[1],mapLabels[row[2]],row[3]])
    f.close()

In [86]:
for x in turb_data_converted:
    if x[2] == 'comeback':  
        print(x)

In [65]:
# Iterate through fDOM and add each each anomalous data point to its corresponding lists 
tilted_peaks = []
single_point_peaks = []
three_peak_violations = []
skyrocketing_peaks = []
plummeting_peaks = []
normal_peaks = []
fluctuations = []

tilted_peak_label = 'question'
single_point_peaks_label = 'single_point'
plummeting_peak_label = 'plummeting'
skyrocketing_label = 'skyrocketing'
normal_peak_label = 'normal_peak'
fluctuation_label = 'fluctuation'
three_peak_label = 'three_peak_vio'

for row in fDOM_data_converted:
    if row[2] == tilted_peak_label:
        tilted_peaks.append(row)
    elif row[2] == single_point_peaks_label:
        single_point_peaks.append(row)
    elif row[2] == plummeting_peak_label:
        plummeting_peaks.append(row)
    elif row[2] == skyrocketing_label:
        skyrocketing_peaks.append(row)
    elif row[2] == normal_peak_label:
        normal_peaks.append(row)
    elif row[2] == fluctuation_label:
        fluctuations.append(row)
    elif row[2] == three_peak_label:
        three_peak_violations.append(row)


turb_spike_anomalies = turb_data # turb data only has one anomaly type currently 



In [66]:
print(len("Tilted Peaks: {}".format(len(tilted_peaks))))
print(len("Single Point Peaks Peaks: {}".format(len(single_point_peaks))))
print(len("Plummeting Peaks: {}".format(len(plummeting_peaks))))
print(len("Normal Peaks: {}".format(len(normal_peaks))))
print(len("Skyrocketing Peaks: {}".format(len(skyrocketing_peaks))))
print(len("Local Fluct: {}".format(len(fluctuations))))
print(len("Three Peak Vio: {}".format(len(three_peak_violations))))


17
28
20
18
22
16
18


In [67]:
# Determine the start and end of each separate anomaly; append each [start, end, type] tuple to new lists 

# This determines how much time we use as a buffer to label starts and ends of event
delta15 = datetime.timedelta(minutes=15)
delta30 = datetime.timedelta(minutes=30)
"""
Take in list of data consisting of data points labeled to be part of an anomaly and 
determine the start and end point of each anomaly 

param: data - form: [datetime,value,type]
return: list of anomalies in form: [start_datetime, end_datetime, type]
"""
def delineate_anomalies(data):
    print(data[0][2])
    anomalies = []
    if len(data) < 3:
        return anomalies 
    starts = []
    ends = []
    # starts of anomalies will not have a point x minutes before 
    # ends will not have a point x minutes after 
    starts.append(data[0][0])
    for i in range(2,len(data)): # start at two as an event is at minumum three points and we have already considered 0 a start 
        # Here we allow index errors
        try: 
            # If the previous point is more than 15 minutes behind the current point then we can 
            # assume that a non anomalous point separates them and that this point is a start 
            # a start cannot occur in the last 2 entries as this would not allow for a complete anomaly 
            if i<=(len(data)-3) and data[i-1][0] + delta15 < data[i][0] and data[i-2][0]+delta30 < data[i][0]:
                starts.append(data[i][0])
            elif data[i+1][0] - delta15 > data[i][0] and data[i+2][0] - delta30 > data[i][0]:
                ends.append(data[i][0])
        except IndexError: 
            # print('Error: i = {}, out of {} rows'.format(i,len(data)))
            pass
    ends.append(data[len(data)-1][0]) # We will get an error when calculating an end point at the end of the list so we append it here
    # Verify Starts : Each start should have at least 2 more consecutive points 
    for i in range(len(data)):
        if data[i][0] in starts:
            # A start should not occur in the last 2 indices of the data so this should not produce errors
            if not ((data[i][0] == data[i+1][0] - delta15) or (data[i][0] == data[i+2][0] - delta30)):
                print('Error: Starts are not correct and do not have two perfect consecutive points...')
                print('Index: {}, Start: {}'.format(i, data[i][0]))
    
    # There should be an equal number of starts and ends
    if len(starts) != len(ends):
        print('Error: Starts = {}, Ends = {}\n'.format(len(starts), len(ends)))
        print('Starts:')
        for val in starts:
            print(val)
        print('\nEnds:')
        for val in ends: 
            print(val)
        return anomalies
    # If there are an equal number of starts and ends then the i entries in starts and ends should be from the same anomaly
    for i in range(len(starts)):
        anomalies.append([starts[i],ends[i],data[0][2]])
    return anomalies


In [68]:
# For each anomaly type list: determine start and ends of each separate anomaly occurence and append to new list: 
processed_tilted_peaks = delineate_anomalies(tilted_peaks)
processed_single_point_peaks = delineate_anomalies(single_point_peaks)
processed_three_peak_violations = delineate_anomalies(three_peak_violations)
processed_skyrocketing_peaks = delineate_anomalies(skyrocketing_peaks)
processed_plummeting_peaks = delineate_anomalies(plummeting_peaks)
processed_normal_peaks = delineate_anomalies(normal_peaks)
processed_fluctuations = delineate_anomalies(fluctuations)


question
single_point
three_peak_vio
skyrocketing
plummeting
normal_peak
fluctuation


In [69]:
# Function takes outfile name, data to write, and peak type label, and writes data out to csv file 
def write_data_to_csv(out_file, data, peak_type):
    with open(out_file, 'w',newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter = ',')
        csvwriter.writerow(['start_time','end_time','peak_type'])
        for row in data:
            csvwriter.writerow([str(row[0]), str(row[1]), peak_type])
    csvfile.close()

In [70]:
# write out data to txt files: [start_datetime, end_datetime, anomaly_type]
base_path = '../Data/data_annotating/processed_annotated/10.1.12-7.13.20/'
write_data_to_csv(base_path + 'plummetting_peaks.csv', processed_plummeting_peaks, 'plummeting_peak')
write_data_to_csv(base_path + 'tilted_peaks.csv',  processed_tilted_peaks, 'tilted_peaks')
write_data_to_csv(base_path + 'single_point_peaks.csv', processed_single_point_peaks, 'single_point_peaks')
write_data_to_csv(base_path + 'three_peak_violations.csv',processed_three_peak_violations,'three_peak_violation')
write_data_to_csv(base_path + 'skyrocketing_peaks.csv', processed_skyrocketing_peaks, 'skyrocketing_peak')
write_data_to_csv(base_path + 'normal_peaks.csv', processed_normal_peaks, 'normal_peak')
write_data_to_csv(base_path + 'local_fluctuations.csv', processed_fluctuations, 'local_fluctuations')


In [34]:
len(processed_skyrocketing_peaks)

3